<a href="https://colab.research.google.com/github/lucas-labs/proyeccion-ballotage-argentina-23/blob/master/collab_proyeccion_ballotage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyección Elecciones

## Imports

In [9]:
import plotly.graph_objs as go
import pandas as pd

## Distribución de votos

Teniendo en cuenta los resultados de las elecciones y suponiendo que 50% de los votos de Schiaretti y Bullrich irían a Milei, 30% de los mismos a Massa y 20% en blanco; sumando 70% de los votos de la Izquierda a Massa.

In [10]:
distribution_map = {
    #   Massa, Milei, Bullr, Schiar, Breg
    5: [1.0,   0.0,   0.3,   0.3,    0.7], # Massa
    6: [0.0,   1.0,   0.5,   0.5,    0.0], # Milei
    7: [0.0,   0.0,   0.2,   0.2,    0.3], # Blanco/Nulo
}

#### Calculo de predicciones

In [11]:
# los votos se encuentran en la columna 'Value
votos = pd.read_csv('nodes.csv')

# calculate Value in millions create a column Value_M
# that is the Value column divided by 1 million and rounded to 2 decimals
votos['Value_M'] = round(votos['Value'] / 1000000, 2)

# set label as Label + (Value_M) but only  if Value_M is greater than 0 and it's  not NaN
# votos['Label'] = votos['Label'] + ' (' + votos['Value_M'].astype(str) + 'M)'
votos['Label'] = votos.apply(lambda row: row['Label'] + ' (' + str(row['Value_M']) + 'M)' if row['Value_M'] > 0 else row['Label'], axis=1)

link_colors = {
    0: { # from Massa to
        5: 'rgba(0, 156, 222, 0.7)', # Massa
        6: 'rgba(59, 108, 206, 0.7)', # Milei
        7: 'rgba(120, 201, 236, 0.7)', # Blanco/Nulo
    },
    1: { # from Milei to
        5: 'rgba(59, 108, 206, 0.7)', # Massa
        6: 'rgba(117, 59, 189, 0.7)', # Milei
        7: 'rgba(178, 153, 220, 0.7)', # Blanco/Nulo
    },
    2: { # from Bullrich to
        5: 'rgba(127, 189, 111, 0.7)', # Massa
        6: 'rgba(163, 113, 126, 0.7)', # Milei
        7: 'rgba(247, 234, 125, 0.7)', # Blanco/Nulo
    },
    3: { # from Schiaretti to
        5: 'rgba(34, 114, 183, 0.7)', # Massa
        6: 'rgba(92, 66, 166, 0.7)', # Milei
        7: 'rgba(153, 159, 197, 0.7)', # Blanco/Nulo
    },
    4: { # from Bregman to
        5: 'rgba(125, 120, 160, 0.7)', # Massa
        6: 'rgba(183, 72, 143, 0.7)', # Milei
        7: 'rgba(244, 165, 174, 0.7)', # Blanco/Nulo
    }
}

# creamos un df con el siguiente formato:
# Source, Target, Value
# donde Source y Target son los ids de los nodos
# y Value es la cantidad de votos que se transfieren de Source a Target

prediction = pd.DataFrame(columns=['Source', 'Target', 'Value', 'Link_Color'])

# target es el key del map
for target in distribution_map:
    distribution = distribution_map[target]
    for source, percentage in enumerate(distribution):
        # agarro el valor de votos del source desde el df de votos
        source_value = votos.iloc[source]['Value']
        color = link_colors[source][target]
        val = source_value * percentage
        row = [source, target, val, color]
        prediction.loc[len(prediction)] = row

## Proyección

### Config

In [12]:
import plotly.graph_objects as go

fig = go.Figure(
    data=[go.Sankey(
        type='sankey',
        domain = dict(
            x =  [0,1],
            y =  [0,1]
        ),
        orientation = "h",
        valueformat = ".0f",
        node = dict(
            pad = 10,
            thickness = 30,
            line = dict(
                color = "black",
                width = 0.5
            ),
            label =  votos['Label'].dropna(axis=0, how='any'),
            color = votos['Color']
        ),
        link = dict(
            source = prediction['Source'].dropna(axis=0, how='any'),
            target = prediction['Target'].dropna(axis=0, how='any'),
            value = prediction['Value'].dropna(axis=0, how='any'),
            color = prediction['Link_Color'].dropna(axis=0, how='any'),

        ),

    )],
    layout = go.Layout(
        title = "Transferencia de Votos para Ballotage",
        height = 772,
        width = 950,
        font=dict(
            size = 14,
        )
    )
)

### Chart

In [13]:
fig.show()

## Totales

### Config

In [15]:
# Create a bar chart with the totals, using Percentage column
# set the Value_M on votes for targets by grouping by Target and summing the Value
# and formatting the Value_M to 2 decimals
totals = prediction.groupby('Target').sum()
totals['Value_M'] = round(totals['Value'] / 1000000, 2)

# create a new Percentage Col for totals, calculate the percentage of each row by its Value  column (not Value_M)
# and format it to 2 decimals
totals['Percentage'] = round(totals['Value'] / totals['Value'].sum() * 100, 2)

# Calculate Percentage not blank column by calculating the percentage of each row by its Value column but
# excluding the the 3rd row (which are the blank votes) from  the total sum. This means that the 3rd row will
# always be 0% as it doen't count for the calculation
totals['Percentage_without_blanks'] = round(totals['Value'] / (totals['Value'].sum() - totals.iloc[2]['Value']) * 100, 2)
totals.loc[7, 'Percentage_without_blanks'] = 0

# set totals['Label'] using the label of the target from votos
totals['Label'] = totals.apply(lambda row: votos.iloc[row.name]['Label'], axis=1)

# now for each target in totals set the label on votes with:
# Label + (Value_M)
for target in totals.index:
    label = votos.iloc[target]['Label']
    value_m = totals.loc[target]['Value_M']
    votos.loc[target, 'Label'] = label + ' (' + str(value_m) + 'M)'

def get_layout(title):
    return go.Layout(
        height = 500,
        width = 500,
        font=dict(
            size = 14,
        ),
        title=title
    )

fig = go.Figure(
    data=[go.Bar(
        x=totals['Percentage'],
        y=totals['Label'],
        orientation='h',
        marker=dict(
            color=votos['Color']
        )
    )],
    layout = get_layout('Resultados Ballotage')
)

# and another bar chart with the totals, using Percentage_without_blanks column
fig2 = go.Figure(
    data=[go.Bar(
        x=totals['Percentage_without_blanks'],
        y=totals['Label'],
        orientation='h',
        marker=dict(
            color=votos['Color']
        )
    )],
    layout = get_layout('Resultados Ballotage sin Blancos/Nulos')
)



<ipython-input-15-d4db8d0667f4>:4: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



### Chart

In [16]:
fig.show()
fig2.show()